# 3. DOMAĆA ZADAĆA - AK.GOD 2017/2018

In [1]:
from math import sqrt,sin
import numpy as np
import simple_optimisation
from LUP import Matrix

## Postupak najbržeg spusta (gradijentni spust)

In [2]:
def gradient_descent(f,x0,golden_ratio=False,e=10e-6):
    x = np.asarray(x0)
        
    gradient = np.asarray(f.gradient(x0))
    gradient_norm = np.linalg.norm(gradient)
    iteration = 0
    
    while(gradient_norm > e):
        v = [-i/gradient_norm for i in gradient]
        
        for i in range(len(x0)):
            vi = [ v[j] if i==j else 0 for j in range(len(v))]
            lambda_ = lambda alpha: f.call([x[j] + alpha*vi[j] for j in range(len(x0))])
            lambda_min = 1 if not golden_ratio else simple_optimisation.golden_ratio(f=lambda_, starting_point=x[i])
            x = x+lambda_min*np.asarray(vi)
        
        gradient_norm_old = gradient_norm
        gradient = np.asarray(f.gradient(x.tolist()))
        gradient_norm = np.linalg.norm(gradient)
        
        if gradient_norm_old < gradient_norm:
            iteration += 1
        
        if iteration >= 100:
            print("Ended at max same iterations")
            break
        print(gradient_norm)
    print(x) 
    return x

In [3]:
class Function:
    counter = 0
    def __init__(self,f,df,H=None):
        self.f = f
        self.df = df
        if H: self.H = H
    
    def call(self,x):
        self.counter += 1
        return self.f(x)
    
    def gradient(self,x):
        return self.df(x)
    
    def hesse(self,x):
        return self.H(x)
    
    def gradient_norm(self,x):
        result = 0
        for i in self.df(x):
            result += i**2
        return sqrt(result)

In [4]:
function = lambda x: 100*(x[1]-x[0]**2)**2 + (1-x[0])**2
dfunction = lambda x: [-400*x[0]*(x[1]-x[0]**2)-2*(1-x[0]),200*(x[1]-x[0]**2)]
def hesse(x):
    return [[-400*(x[1]-3*x[0]**2)+2, -400*x[0]],[-400*x[0], 200]]
    
f1 = Function(function,dfunction,H=hesse)
function(gradient_descent(f1,[-1.9,2],golden_ratio=True))

4.82239849918
0.821343292725
4.81527667295
0.814248112484
4.80815748283
4.80204943462
4.79592226632
4.78977564913
4.78360989106
4.77742470462
4.77122035657
4.76499097234
4.75874588441
4.75247926007
4.74619261095
4.73988147511
4.73355908723
4.72720597793
4.72083819017
4.71444547938
4.70803703571
4.70159814596
4.69514499905
4.68866442248
4.68216644888
4.67564393805
4.66910064272
4.66252610111
4.65593202622
4.64931275325
4.64267686939
4.63600911284
4.62932391459
4.62260524989
4.61587151729
4.60910913343
4.60232087925
4.59550628064
4.58867250702
4.58180548357
4.57491451235
4.56799555129
4.5610574644
4.55407940755
4.54708551712
4.54005268314
4.533002518
4.52591959316
4.51880779017
4.5116640487
4.50449299925
4.49729322372
4.49006338577
4.48279938494
4.47551258987
4.46818239902
4.46083123858
4.45344388415
4.44602604617
4.43857342586
4.4310874415
4.42357009629
4.41601704722
4.40842455526
4.40080181187
4.39314149995
4.38544765115
4.37771669718
4.36995305596
4.3621449795
4.35430769127
4.34642604

0.00595615785348
0.00594340271319
0.00593262178132
0.00592017407924
0.00590090062333
0.00588620364874
0.00587153941668
0.00585695328178
0.00584598962139
0.00582969721516
0.0058153209386
0.00580131069827
0.00578801999008
0.00577618087245
0.00576160375808
0.00574719192993
0.00573309789182
0.00572313093182
0.00570881928748
0.00569493873991
0.00567627860744
0.00566280809582
0.00565059667434
0.00563528763487
0.00561939922061
0.00560236474154
0.00559213296235
0.0055807830339
0.00556370548654
0.00554777185641
0.00553626258803
0.00552235760233
0.00550927858129
0.0054922324614
0.00548197910774
0.00546710561075
0.00545556873677
0.00543812833913
0.00542708649783
0.0054141145873
0.00540290229269
0.00538958990339
0.00537769638018
0.00536865008945
0.0053596604111
0.00534731409515
0.00533172432256
0.00531528544058
0.00530624633688
0.00529728034838
0.0052849725394
0.00526946867424
0.00525319244785
0.00524448799181
0.00522707462768
0.00521609584359
0.00519976238534
0.00518531477738
0.00517115107538
0.0

4.6976455171732879e-08

In [5]:
function = lambda x: (x[0]-4)**2 + 4*(x[1]-2)**2
dfunction = lambda x: [2*(x[0]-4),8*(x[1]-2)]
f2 = Function(function,dfunction)
function(gradient_descent(f2,[0.1,0.3],golden_ratio=True))

1.10895615653e-06
[ 3.99999956  2.00000008]


2.2426653203457529e-13

## Newton-Raphsonov postupak

In [45]:
def lin_system(A,b):
    dek, P = A.LUP_decomposition()
    L = dek.get_L()
    U = dek.get_U()
    y = L.forward_substitution(P*b)
    x = U.back_substitution(y)
    return x

def set_parameters(f,x):
    df_ = np.asarray([f.gradient(x)]).T.tolist()
    dF = Matrix(len(x),1)
    dF.set_data(df_)
    H = Matrix(len(x),len(x))
    H.set_data(f.hesse(x))
        
    dx = np.asarray(lin_system(H,-1*dF).tolist()).T
    v = dx/np.linalg.norm(dx)
    return v

In [52]:
def newton_raphson(f,x0,golden_ratio=False,e=10e-6):
    x = np.asarray(x0)
    v = set_parameters(f,x).tolist()[0]
    
    iteration = 0
    
    while(np.linalg.norm(v) > e):
        x_old = x.copy()
        for i in range(len(x0)):
            vi = [ v[j] if i==j else 0 for j in range(len(v))]
            lambda_ = lambda alpha: f.call([x[j] + alpha*vi[j] for j in range(len(x0))])
            lambda_min = 1 if not golden_ratio else simple_optimisation.golden_ratio(f=lambda_, starting_point=x[i])
            x = x+lambda_min*np.asarray(vi)
        
        v = set_parameters(f,x).tolist()[0]
        
        if f.call(x) > f.call(x_old):
            iteration += 1
        
        if iteration > 100:
            break
            
    print(x)
    return x

In [55]:
f1.call(newton_raphson(f1,[-1.9,2],golden_ratio=True))

[ 0.99984535  0.99969146]


2.3970411160557376e-08